### 首先是数据集
MNIST 手写数据集太老了，可以选择Fashion-MNIST 数据集

### 手写Softmax 分类问题的收获
- 学习了 torchvision.transforms 的一些操作，主要是对图片数据的处理
    - 例如 ToTensor() plt.imgshow()也能接受 tensor变量 还有使用管道对数据进行批量处理 transforms.Compose = [] Resize 调整大小啊
- 大体了解了一下DataSet这个类，自己写MyDataSet 要继承这个基类，然后重写 __getitem__ __len__ 这两个函数 __getitem() 返回一个x，一个y
- DataLoader(dataset = ,bitch_size= , shuffle = True,num_workers = )
- 写softmax函数之前，详细了解了花式索引 y_hat[[0,1],[0,2]]
- 会了一行代码完成交叉熵损失函数
- 学会如何计算模型的准确率
    - n个变量的累加器 Accumulator
    - 使用bool列表 然后求和 求准确率 cmp = y_hat.type(y.dtype) == y
    

In [36]:
%matplotlib inline
import torch
import torchvision
from torch.utils import data
import matplotlib.pyplot as plt
from torchvision import transforms
import numpy as np
from d2l import torch as d2l

d2l.use_svg_display() # 使用svg来显示图片

### 定义我们的加载数据集函数

In [37]:
from random import shuffle


def load_data_fashion_mnist(batch_size ,resize = None):
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0,transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root = '../data',train = True,transform = trans,download = True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root = '../data',train = False,transform = trans,download = True)
    return (data.DataLoader(mnist_train,batch_size,shuffle = True,
                num_workers = 4),
            data.DataLoader(mnist_test,batch_size,shuffle = True,num_workers = 4))


In [38]:
batch_size = 256
train_iter,test_iter = load_data_fashion_mnist(batch_size)

In [39]:
print(next(iter(train_iter)))

[tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  .

In [40]:
# 初始化模型参数 这是一个一层的网络结构
num_inputs = 28*28
num_outputs = 10 # 10分类
w = torch.normal(0,0.01,size = (num_inputs,num_outputs),requires_grad = True)
b = torch.zeros(num_outputs,requires_grad= True)

In [41]:
# 定义softmax操作

def softmax(x):
    x_exp = torch.exp(x)
    partition = x_exp.sum(dim = 1,keepdim = True)
    # 这里为什么选择第一个维度，是因为 x的size 是 (batch_size,num_features)
    # 也就是 (batch_size,10) 因为经过矩阵的乘法 (batch_size,28*28) * (28*28,10) 已经变成了(batch_sizem,10)
    return x_exp / partition # 利用了广播机制
    # partition的机构应该是 (batch_size,1)  

In [42]:
# 定义网络模型
def net(x):
    return softmax(torch.matmul(x.reshape(-1,w.shape[0]),w) + b)


In [43]:
# 计算交叉熵时，通常会使用到 Tensor张量的一种特殊索引类型 花式索引 Fancy indexing
# 也就是使用列表 给出下标来确定位置
# 单维度
# import torch
x = torch.tensor([10, 20, 30, 40])
indices = torch.tensor([0, 2])
print(x[indices])  # 输出: tensor([10, 30])
# 多维度
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
"""
y_hat[[0, 1], [0, 2]] 是在选择了第一行和第二行的基础上，
对第一行选择第一列，第二行选择第二列   是在选择的基础上在进行选择
y_hat[:,y] 利用了广播机制， 首先选了第一行和第二行，然后对两个行都选择第一列和第三列
"""
y_hat[[0, 1], y],y_hat[:,y]

tensor([10, 30])


(tensor([0.1000, 0.5000]),
 tensor([[0.1000, 0.6000],
         [0.3000, 0.5000]]))

In [44]:
# 定义损失函数 交叉熵损失函数

def cross_entropy(y_hat,y):
    return -torch.log(y_hat[range(len(y)),y])
cross_entropy(y_hat,y)

tensor([2.3026, 0.6931])

In [45]:
# 计算正确率 accuracy 计算当前这一批次 的准确率
def accuracy(y_hat,y):
    """
        传入格式 y_hat (batch_size,num_class)
        第一维度是每个样本，第二维度是每个样本对每一类的预测
        y 则 是(batch_size) 即每一个样本所属的类
    """
    if len(y_hat) > 1 and y_hat.shape[1] > 1:
        # 保证这个批次不只有一个样本，且class 大于1
        y_hat = y_hat.argmax(axis = 1)
    cmp = y_hat.type(y.dtype) == y
    # cmp 是一个bool 数组 相等的地方为true 其他未False
    return float(cmp.type(y.dtype).sum())
    # 返回预测正确的个数

In [46]:
accuracy(y_hat, y) / len(y)

0.5

In [47]:
class Accumulator:
    def __init__(self,n):
        self.data = [0.0] * n
    
    def add(self,*args):
        self.data = [a + float(b) for a,b in zip(self.data,args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self,idx):
        return self.data[idx]



In [48]:
# 评估
"""计算在指定数据集上模型的精度"""
def evaluate_accuracy(net,data_iter):
    # 传入网络和数据集
    if isinstance(net,torch.nn.Module):
        net.eval() # 将模型设置为评估模式

    metric = Accumulator(2) # 表示有两个变量 正确的和总量

    for x,y in data_iter:
        metric.add(accuracy(net(x),y),y.numel())
    
    return metric[0] / metric[1]

    

In [ ]:
evaluate_accuracy(net,test_iter) # 算一个随机的初始化的精度
# 正确率只有 8.7%

0.087

### 手写Softmax 分类问题的收获
- 学习了 torchvision.transforms 的一些操作，主要是对图片数据的处理
    - 例如 ToTensor() plt.imgshow()也能接受 tensor变量 还有使用管道对数据进行批量处理 transforms.Compose = [] Resize 调整大小啊
- 大体了解了一下DataSet这个类，自己写MyDataSet 要继承这个基类，然后重写 __getitem__ __len__ 这两个函数 __getitem() 返回一个x，一个y
- DataLoader(dataset = ,bitch_size= , shuffle = True,num_workers = )
- 写softmax函数之前，详细了解了花式索引 y_hat[[0,1],[0,2]]
- 会了一行代码完成交叉熵损失函数
- 学会如何计算模型的准确率
    - n个变量的累加器 Accumulator
    - 使用bool列表 然后求和 求准确率 cmp = y_hat.type(y.dtype) == y
    

print("hello world")

In [50]:
print("hello world")

hello world
